<a href="https://colab.research.google.com/github/EmidioLP/Freecodecamp/blob/main/Notebooks/Machine%20Learning%20with%20Python/RNN_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esse notebook é baseado nas aulas disponibilizadas em: https://www.freecodecamp.org/portuguese/learn/machine-learning-with-python/

Esse notebook é baseado no seguinte guia disponibilizado na documentação do Tensorflow: https://www.tensorflow.org/tutorials/text/text_generator

#Imports e Setup

In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

# Dataset

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


# Loading dos dados

In [ ]:
#Para escolher um arquivo pessoal no lugar de fazer o download
from google.colab import files
path_to_file = list(files.upload().keys())[0]

In [5]:
#Lê e depois decodifica para o formato py2
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# "Lenght of text" é o número de caracteres contidos nele
print('Lenght of text: {} characters'.format(len(text)))

Lenght of text: 1115394 characters


In [6]:
#Os primeiros 250 caracteres no texto
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



# Codificando

Como o texto não está codificado em números (que é como o modelo de IA precisa receber para entendê-lo), precisa-se fazer isso manualmente.

In [7]:
vocab = sorted(set(text))
# Mapeando caracteres únicos em indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [8]:
print("Texto: ", text[:13])
print("Codificado: ", text_to_int(text[:13]))

Texto:  First Citizen
Codificado:  [18 47 56 57 58  1 15 47 58 47 64 43 52]
